In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
import cv2

def extract_sprites_horizontal(sprite_sheet_paths, num_frames, output_dir):
    # Check if output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for sprite_sheet_path in sprite_sheet_paths:
        sprite_sheet = cv2.imread(sprite_sheet_path)
        if sprite_sheet is None:
            print(f"Failed to load sprite sheet: {sprite_sheet_path}")
            continue

        # Get the dimensions of the sprite sheet
        sheet_height, sheet_width, _ = sprite_sheet.shape
        frame_width = sheet_width // num_frames

        # Extract each frame
        for i in range(num_frames):
            frame = sprite_sheet[:, i * frame_width:(i + 1) * frame_width]
            frame_filename = os.path.join(output_dir, f"frame_{i+1}.png")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved frame {i+1} to {frame_filename}")

# Example usage
sprite_sheet_paths = ['./Walk_City_men_1_10f.png']  # Path to your sprite sheet
num_frames = 10  # Total number of frames in the sprite sheet
output_dir = "output_frames"  # Directory to save the frames

extract_sprites_horizontal(sprite_sheet_paths, num_frames, output_dir)

In [ ]:
import os
import cv2

# Step 1: Load frames from the output directory
def load_frames_from_dir(output_dir):
    frames = []
    frame_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])

    for file in frame_files:
        frame_path = os.path.join(output_dir, file)
        frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale for optical flow
        frames.append(frame)

    return frames

# Example usage
output_dir = "output_frames"
frames = load_frames_from_dir(output_dir)

# Step 2: Compute motion vectors (Optical Flow)
motion_vectors = []
prev_frame = frames[0]

for i in range(1, len(frames)):
    next_frame = frames[i]

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Append flow (motion vectors) for each frame
    motion_vectors.append(flow)

    prev_frame = next_frame

# Now, you have motion_vectors ready for further use


In [ ]:
# We will use Farneback Optical Flow to estimate motion vectors between consecutive frames
# Parameters can be tuned to better match the small movement between sprite frames

motion_vectors = []
prev_frame = frames[0]

# Compute optical flow between each consecutive frame
for i in range(1, num_frames):
    next_frame = frames[i]

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Append flow (motion vectors) for each frame
    motion_vectors.append(flow)

    prev_frame = next_frame

# Let's visualize the motion vectors for the first frame transition to confirm the results
magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

# Create a blank image to visualize the flow
hsv = np.zeros_like(cv2.cvtColor(frames[0], cv2.COLOR_GRAY2BGR))
hsv[..., 1] = 255

# Use the magnitude and angle to create a visualization
hsv[..., 0] = angle * 180 / np.pi / 2
hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

# Convert to BGR for display
flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Display the motion vector visualization
plt.imshow(flow_rgb)
plt.title("Optical Flow between Frame 1 and Frame 2")
plt.axis("off")
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Step 1: Preparing the data

class SpriteDataset(Dataset):
    def __init__(self, frames, motion_vectors):
        self.frames = frames
        self.motion_vectors = motion_vectors

    def __len__(self):
        return len(self.motion_vectors)

    def __getitem__(self, idx):
        first_frame = self.frames[idx]  # Input: first frame
        motion_vector = self.motion_vectors[idx]  # Input: motion vector between frame idx and idx+1
        next_frame = self.frames[idx + 1]  # Target: the next frame to be generated

        # Normalize data for the network (float conversion for PyTorch tensors)
        first_frame = torch.tensor(first_frame).unsqueeze(0).float() / 255.0
        next_frame = torch.tensor(next_frame).unsqueeze(0).float() / 255.0
        motion_vector = torch.tensor(motion_vector).float()

        return first_frame, motion_vector, next_frame

# Create the dataset
sprite_dataset = SpriteDataset(frames, motion_vectors)
data_loader = DataLoader(sprite_dataset, batch_size=1, shuffle=True)

# Step 2: Defining the GAN structure

# Generator: takes the first frame and motion vector as input and generates the next frame
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(128*128 + 128*128*2, 1024)
        self.fc2 = nn.Linear(1024, 128*128)

    def forward(self, frame, motion_vector):
        # Flatten the frame and motion vector
        frame = frame.view(frame.size(0), -1)
        motion_vector = motion_vector.view(motion_vector.size(0), -1)

        # Concatenate frame and motion vector
        x = torch.cat([frame, motion_vector], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))

        # Reshape back to image format
        x = x.view(-1, 1, 128, 128)
        return x

# Discriminator: takes the frame and the next frame, and predicts whether the next frame is real or generated
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(128*128*2, 1024)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, frame, next_frame):
        # Flatten the frames
        frame = frame.view(frame.size(0), -1)
        next_frame = next_frame.view(next_frame.size(0), -1)

        # Concatenate the frames
        x = torch.cat([frame, next_frame], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Step 3: Initializing the models, loss function, and optimizers

generator = Generator()
discriminator = Discriminator()

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for GANs
# Example of using MSELoss instead of BCELoss
# criterion = nn.MSELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0001)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0001)

# Step 4: GAN training loop

def train_gan(epochs=100):
    for epoch in range(epochs):
        for i, (first_frame, motion_vector, real_next_frame) in enumerate(data_loader):
            # Train the discriminator
            optimizer_d.zero_grad()

            # Real frame pair (label=1)
            real_label = torch.ones((first_frame.size(0), 1))
            real_output = discriminator(first_frame, real_next_frame)
            real_loss = criterion(real_output, real_label)

            # Generated frame pair (label=0)
            fake_next_frame = generator(first_frame, motion_vector)
            fake_label = torch.zeros((first_frame.size(0), 1))
            fake_output = discriminator(first_frame, fake_next_frame.detach())
            fake_loss = criterion(fake_output, fake_label)

            # Total discriminator loss
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train the generator
            optimizer_g.zero_grad()
            fake_output = discriminator(first_frame, fake_next_frame)
            g_loss = criterion(fake_output, real_label)  # We want the generator to fool the discriminator
            g_loss.backward()
            optimizer_g.step()

        # Logging the losses for both Generator and Discriminator
        print(f"Epoch [{epoch+1}/{epochs}] | D_loss: {d_loss.item()} | G_loss: {g_loss.item()}")

# Start training the GAN
train_gan(epochs=100)  # Using a small number of epochs for demonstration purposes

In [ ]:
import matplotlib.pyplot as plt

# Example: Generate images using the trained generator
def generate_images(generator, first_frame, motion_vectors):
    generated_sequence = []
    current_frame = first_frame

    for motion_vector in motion_vectors:
        # Ensure motion_vector is correctly converted to tensor and has the correct shape
        motion_vector = torch.tensor(motion_vector).unsqueeze(0).float()

        # Optional: Print shapes to debug
        print(f"Current frame shape: {current_frame.shape}")
        print(f"Motion vector shape: {motion_vector.shape}")

        with torch.no_grad():
            next_frame = generator(current_frame, motion_vector)
            generated_sequence.append(next_frame.squeeze().cpu().numpy() * 255)  # Store generated frames

        current_frame = next_frame  # Update current frame for the next iteration

    return generated_sequence

# Test generation using first frame and motion vectors
first_frame = torch.tensor(frames[0]).unsqueeze(0).unsqueeze(0).float() / 255.0
generated_frames = generate_images(generator, first_frame, motion_vectors)


In [ ]:
def display_generated_frames_horizontal(frames):
    num_frames = len(frames)

    # Create a horizontal figure with a subplot for each frame
    fig, axes = plt.subplots(1, num_frames, figsize=(num_frames * 2, 4))  # Adjust figsize for wider output

    for idx, frame in enumerate(frames):
        axes[idx].imshow(frame, cmap='gray')
        axes[idx].set_title(f"Frame {idx + 1}")
        axes[idx].axis("off")  # Hide axes

    plt.tight_layout()
    plt.show()

# Call the function to display frames horizontally
display_generated_frames_horizontal(generated_frames)

In [ ]:
first_frame = torch.tensor(frames[0]).unsqueeze(0).unsqueeze(0).float() / 255.0
print(first_frame.shape)  # 입력된 첫 번째 프레임의 크기 확인


In [ ]:
import matplotlib.pyplot as plt

# Example: Visualize the generated frame vs real frame
def visualize_frames(first_frame, real_next_frame, generated_frame):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # First frame
    axes[0].imshow(first_frame.squeeze().cpu().numpy(), cmap='gray')
    axes[0].set_title("First Frame")

    # Real next frame
    axes[1].imshow(real_next_frame.squeeze().cpu().numpy(), cmap='gray')
    axes[1].set_title("Real Next Frame")

    # Generated frame - detach before converting to numpy
    axes[2].imshow(generated_frame.squeeze().detach().cpu().numpy(), cmap='gray')
    axes[2].set_title("Generated Frame")

    plt.show()

# Example: Generate frames and visualize
first_frame, motion_vector, real_next_frame = next(iter(data_loader))
generated_frame = generator(first_frame, motion_vector)

visualize_frames(first_frame, real_next_frame, generated_frame)


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
def animate_frames(predicted_frames):
    """
    Create and display an animation from predicted frames in Jupyter Notebook.

    Parameters:
    predicted_frames (numpy array): Array of predicted frames, expected shape (num_frames, 128, 128, 3).
    """
    num_frames = predicted_frames.shape[0]

    # Create a figure
    fig, ax = plt.subplots()

    # Display the first frame initially
    img = ax.imshow(predicted_frames[0], animated=True, cmap='gray')
    ax.axis('off')  # Hide the axis for visual appeal

    # Update function for the animation
    def update(frame_num):
        img.set_array(predicted_frames[frame_num])  # Update the image data with the new frame
        return [img]

    # Create the animation: FuncAnimation creates a new image every interval
    ani = FuncAnimation(
        fig, update, frames=num_frames, interval=200, blit=True  # interval sets the frame display time (in ms)
    )

    # Display the animation in Jupyter
    plt.close(fig)  # Close the figure to prevent a static image from being displayed
    return HTML(ani.to_jshtml())  # Display the animation in HTML format

In [ ]:
animate_frames(np.array(generated_frames))